In [1]:
import os
import random
from datetime import datetime
from collections import Counter

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from utils import call_backs, misc, preprocessing
from custom_layers.arcface_loss import ArcMarginProduct

### Config

In [2]:
class config:
    # GENERAL
    RANDOM_SEED = 5
    TENSOR_LOG_DIR = 'logs'
    SAVE_DIR = 'saved_models'

    # DATA
    INPUT_SIZE = (28,28,1)
    NUM_CLASSES = 10

    # MODEL
    OUTPUT_EMB = 64
    MIDDLE_EMB = 256

    # TRAINING
    BATCH_SIZE = 32
    LR = .000005

misc.seed_everything(config.RANDOM_SEED)

### Load dataset

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = tf.expand_dims(tf.convert_to_tensor(x_train),-1)
y_train = tf.expand_dims(tf.convert_to_tensor(y_train),-1)

In [4]:
class_count = Counter(np.array(tf.reshape(y_train, [60000,])))
print(class_count)

Counter({9: 6000, 0: 6000, 3: 6000, 2: 6000, 7: 6000, 5: 6000, 1: 6000, 6: 6000, 4: 6000, 8: 6000})


In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(preprocessing.normalize).map(preprocessing.arcface_format).batch(config.BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(preprocessing.normalize).map(preprocessing.arcface_format).batch(config.BATCH_SIZE)

### Define model 

In [6]:
# from custom_layers.arcface_loss import ArcMarginProduct
from custom_layers.subcenter_arcface_loss import SubcenterArcMarginProduct as ArcMarginProduct
# allows 2 inputs and 2 outputs

def get_debug_model(s = 10, m = .25):
 #------------------
    # Definition of placeholders
    inp = tf.keras.layers.Input(shape = config.INPUT_SIZE, name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')

    # Definition of layers
    
    #TODO: reasearch filters, get better understanding
    layer_conv1 = tf.keras.layers.Conv2D(filters = 24, kernel_size = (2,2), input_shape = config.INPUT_SIZE, activation ='relu')
    layer_pool1 = tf.keras.layers.MaxPool2D((2,2))
    layer_conv2 = tf.keras.layers.Conv2D(filters = 12, kernel_size = (2,2), activation ='relu')
    layer_pool2 = tf.keras.layers.MaxPool2D((2,2))
    layer_flatten = tf.keras.layers.Flatten()
    layer_dense1 = tf.keras.layers.Dense(config.MIDDLE_EMB)
    # layer_dense2 = tf.keras.layers.Dense(config.NUM_CLASSES)
    layer_arcface = ArcMarginProduct(n_classes=config.NUM_CLASSES, s=s, m=m)
    layer_softmax = tf.keras.layers.Softmax(dtype='float16', name='head_output')

    if config.MIDDLE_EMB != config.OUTPUT_EMB:
        layer_adaptive_pooling = tfa.layers.AdaptiveAveragePooling1D(config.OUTPUT_EMB)
    else:
        layer_adaptive_pooling = tf.keras.layers.Lambda(lambda x: x)  # layer with no operation

    #------------------
    # Definition of entire model
    backbone_output = layer_conv1(inp)
    backbone_output = layer_pool1(backbone_output)
    backbone_output = layer_conv2(backbone_output)
    backbone_output = layer_pool2(backbone_output)
    embed = layer_flatten(backbone_output)
    embed = layer_dense1(embed)
    
    # Training head
    # head_output = layer_dense2(embed)
    head_output = layer_arcface((embed,label))
    head_output = layer_softmax(head_output)
    
    # Inference
    emb_output = layer_adaptive_pooling(embed)

    model = tf.keras.models.Model(inputs = [(inp, label)], outputs = [head_output, emb_output]) # whole architecture

    return model

In [7]:
debug_model = get_debug_model(s=10, m =.4)
debug_model.summary()

(10, None)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inp1 (InputLayer)              [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 27, 27, 24)   120         ['inp1[0][0]']                   
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 13, 13, 24)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 12, 12, 12)   1164        ['max_pooling2d[0][0]']          
                                                                                   

In [8]:
debug_model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = config.LR),
        loss = {'head_output':tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)},
        metrics = {'head_output':[tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)]},
        )

steps_per_epoch = len(train_dataset) // config.BATCH_SIZE  // 20     # "//20" means that the lr is update every 0.1 epoch.
validation_steps = len(test_dataset) // config.BATCH_SIZE
if len(test_dataset) % config.BATCH_SIZE != 0:
    validation_steps += 1
print(steps_per_epoch, validation_steps)

2 10


### Callbacks

In [9]:
# tensorboard

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                        #  write_graph=True,
                        #  write_images=True,
                        update_freq='epoch',
                        #  profile_batch=2,
                        #  embeddings_freq=1
                        )

#emb_callback = call_backs.EmbeddingCallback(x_test, y_test, save_dir = log_dir+'/emb/', embedding_dim = config.OUTPUT_EMB)

### Training

In [10]:
history = debug_model.fit(
        train_dataset,
        epochs=5,
        validation_steps = validation_steps,
        validation_data = test_dataset,
        verbose=1,
        #callbacks=[tensorboard_callback]
    )

Epoch 1/5
(10, None)
(10, None)
1875/1875 [==============================] - 26s 13ms/step - loss: 6.7428 - head_output_loss: 6.7428 - head_output_sparse_categorical_accuracy: 0.0000e+00 - head_output_sparse_top_k_categorical_accuracy: 0.0000e+00 - val_loss: 6.3988 - val_head_output_loss: 6.3988 - val_head_output_sparse_categorical_accuracy: 0.0000e+00 - val_head_output_sparse_top_k_categorical_accuracy: 0.0000e+00
Epoch 2/5
1875/1875 [==============================] - 22s 12ms/step - loss: 6.2654 - head_output_loss: 6.2654 - head_output_sparse_categorical_accuracy: 0.0000e+00 - head_output_sparse_top_k_categorical_accuracy: 0.0000e+00 - val_loss: 6.1340 - val_head_output_loss: 6.1340 - val_head_output_sparse_categorical_accuracy: 0.0000e+00 - val_head_output_sparse_top_k_categorical_accuracy: 0.0000e+00
Epoch 3/5
1875/1875 [==============================] - 23s 12ms/step - loss: 6.0819 - head_output_loss: 6.0819 - head_output_sparse_categorical_accuracy: 0.0000e+00 - head_output_spars

In [11]:
print("Test examples:",len(x_test))
pred_class, pred_emb = debug_model.predict((x_test, y_test))  # I don't like this, fix it (hacky way would be y test of -1s if non given)

# argmax returns largest element's index
pred_class = tf.argmax(pred_class, axis=1)
true_class = y_test

Test examples: 10000
(10, None)
313/313 [==============================] - 2s 6ms/step


In [12]:
confusion_matrix = {}
embedding_data = []

for i in range(config.NUM_CLASSES):
    confusion_matrix[i] = {}
    for ii in range(config.NUM_CLASSES):
        confusion_matrix[i][ii] = 0

from annoy import AnnoyIndex
tree = AnnoyIndex(config.OUTPUT_EMB, 'euclidean')

for i,pred_tensor, y_array, embedding in zip(range(len(true_class)),pred_class,true_class,pred_emb):
    # come out in annoying type
    pred = int(pred_tensor)
    y = int(y_array)

    tree.add_item(i, embedding)

    assert y < config.NUM_CLASSES and pred < config.NUM_CLASSES
    embedding_data.append({'annoy_idx':i,'true_class':y,'pred_class':pred,'embedding':embedding})
    confusion_matrix[y][pred]+=1

In [13]:
# import matplotlib.pyplot as plt
# cm_df = pd.DataFrame(confusion_matrix)

# fig, ax = plt.subplots(figsize=(4,6))
# ax.matshow(cm_df, cmap=plt.cm.Reds, alpha=0.7)
# for i in range(cm_df.shape[0]):
#     for j in range(cm_df.shape[1]):
#         ax.text(x=j, y=i,s=cm_df[i][j], va='center', ha='center', size='large')
 
# plt.xlabel('Predictions', fontsize=18)
# plt.ylabel('Actuals', fontsize=18)
# plt.title('Confusion Matrix', fontsize=18)
# plt.show()

In [14]:
emb_df = pd.DataFrame(embedding_data)
emb_df.head()

,annoy_idx,true_class,pred_class,embedding
0,0,9,5,"[7.1033363, 15.89732, -5.028651, -10.257683, -..."
1,1,2,4,"[9.250469, 12.162376, 15.891676, -9.463962, -3..."
2,2,1,3,"[-3.8861332, -14.644924, -20.623096, 18.95991,..."
3,3,1,3,"[-2.6930714, -12.675936, -32.222614, 20.721607..."
4,4,6,2,"[-3.187432, 6.86291, 3.0866914, -10.591633, 1...."


In [15]:
def dist_to_origin(embedding):
    return np.sqrt(np.dot(embedding,embedding))

emb_df['length']=emb_df['embedding'].apply(dist_to_origin)
emb_df['normed_embeddings']=emb_df['embedding']/emb_df['length']

In [16]:
tree.build(20)

True

In [17]:
def n_neighbors(annoy_idx, neighbor_count=5):
    # the closest embedding is always the same exact embedding, must exclu
    neighbor_count+=1
    return tree.get_nns_by_item(annoy_idx, neighbor_count)[1:]

def neighbor_classes(row, emb_df, true_classes = True):
    if true_classes:
        get_column = 'true_class'
    else:
        get_column = 'pred_class'
    return emb_df.loc[emb_df['annoy_idx'].isin(row.nearest_neighbors),get_column].to_list()
    
def matching_neighbors(row, true_classes = True):
    if true_classes:
        neighbor_class_col = 'neighbor_classes'
        get_column = 'true_class'
    else:
        neighbor_class_col = 'neighbor_pred_classes'
        get_column = 'pred_class'
    return len([True for neighbor_class in row[neighbor_class_col] if row[get_column] == neighbor_class])

emb_df['nearest_neighbors'] = emb_df['annoy_idx'].apply(n_neighbors)
emb_df['neighbor_classes'] = emb_df.apply(lambda row: neighbor_classes(row,emb_df,true_classes=True), axis=1)
emb_df['neighbor_pred_classes'] = emb_df.apply(lambda row: neighbor_classes(row,emb_df,true_classes=False), axis=1)
emb_df['matching_neighbors'] = emb_df.apply(lambda row: matching_neighbors(row,true_classes=True), axis=1)
emb_df['matching_neighbor_preds'] = emb_df.apply(lambda row: matching_neighbors(row,true_classes=False), axis=1)

In [18]:
emb_df.head()

,annoy_idx,true_class,pred_class,embedding,length,normed_embeddings,nearest_neighbors,neighbor_classes,neighbor_pred_classes,matching_neighbors,matching_neighbor_preds
0,0,9,5,"[7.1033363, 15.89732, -5.028651, -10.257683, -...",114.949707,"[0.061795168, 0.13829805, -0.043746535, -0.089...","[9363, 2802, 163, 2874, 5788]","[9, 9, 9, 9, 9]","[5, 7, 5, 5, 5]",5,4
1,1,2,4,"[9.250469, 12.162376, 15.891676, -9.463962, -3...",93.477638,"[0.09895917, 0.13011001, 0.17000511, -0.101243...","[2505, 4150, 3471, 5255, 5465]","[2, 4, 2, 4, 2]","[4, 2, 4, 2, 4]",3,3
2,2,1,3,"[-3.8861332, -14.644924, -20.623096, 18.95991,...",145.679718,"[-0.02667587, -0.10052823, -0.14156464, 0.1301...","[8867, 8874, 2406, 7054, 8400]","[1, 1, 1, 1, 1]","[3, 3, 3, 3, 3]",5,5
3,3,1,3,"[-2.6930714, -12.675936, -32.222614, 20.721607...",144.502258,"[-0.01863688, -0.08772137, -0.22299038, 0.1433...","[2460, 9818, 9427, 1397, 2251]","[1, 1, 1, 1, 1]","[3, 3, 3, 3, 3]",5,5
4,4,6,2,"[-3.187432, 6.86291, 3.0866914, -10.591633, 1....",63.151390,"[-0.050472874, 0.10867393, 0.04887765, -0.1677...","[3277, 8091, 3754, 1867, 3727]","[0, 0, 6, 0, 0]","[2, 2, 2, 2, 2]",1,5


In [19]:
def competition_score(emb_df, neighbor_count):
    # Average of (correct neighbors / looked at neighbors)
    # ex: 1/N * (  sum(  1/neighbor_count * per_item_match_count  )  ) = 1/N * ( 1/neighbor_count * total_match_count)
    total_matches = emb_df['matching_neighbors'].sum()
    score = total_matches / (neighbor_count * len(emb_df))
    return score

In [20]:
competition_score(emb_df,5)

0.75262

In [21]:
emb_df['correct_prediction'] = emb_df['true_class']==emb_df['pred_class']

In [22]:
print(emb_df['correct_prediction'].value_counts())

False    10000
Name: correct_prediction, dtype: int64


In [23]:
emb_df['matching_neighbor_preds'].value_counts()

5    4541
4    1727
3    1351
2    1065
1     786
0     530
Name: matching_neighbor_preds, dtype: int64

In [24]:
emb_df['matching_neighbors'].value_counts()

5    5330
4    1371
3    1012
2     857
1     747
0     683
Name: matching_neighbors, dtype: int64